# Drill - Data Analysis

Analysis of the spectral data obtained for samples of the drill at Mina do Barroso using data obtained with the SciAps commercial device.

In [1]:
%matplotlib widget
from core.experiment import *

# Data table with samples and concentrations

In [2]:
#path to data
root_path = "C:\\Users\\Miguel\\Desktop\\Furo SciAps\\Converted1\\"

#get the concentration data
concentration_data = pd.read_csv(root_path+"Furo.csv",sep=";")
concentration_data.set_index("Sample ID", inplace=True)

concentration_data2 = concentration_data['Chemistry'].groupby('Sample ID').first()
concentration_data1 = concentration_data.groupby('Sample ID').mean()

concentration_data3 = pd.concat([concentration_data1, concentration_data2], axis=1, join='outer', sort=True)
concentration_data3.head(len(concentration_data))

,H,He,Li,Be,B,C,N,O,F,Ne,...,Rg,Cn,Nh,Fl,Mc,Lv,Ts,Og,Li2O,Chemistry
Sample ID,,,,,,,,,,,,,,,,,,,,,
DRI0000001,0,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
DRI0000002,0,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
DRI0000003,0,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
DRI0000004,0,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
DRI0000005,0,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DRI0000125,0,0,11000,108.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,FGP
DRI0000126,0,0,1860,209.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,FGP
DRI0000127,0,0,4280,160.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,FGP


## Load the data from folder

In [3]:
#path to data
root_path = "C:\\Users\\Miguel\\Desktop\\Furo SciAps\\Converted1\\"

#find all folders in path
sample_folders = [f for f in os.listdir(root_path) if os.path.isdir(root_path+f)]

#experiment for each sample
list_of_experiments = []

#sample identification
sample_ids = []
for i in range(0,len(sample_folders)):
    sample_f = sample_folders[i]
    print(" Reading Folder "+ sample_f + " - "+ str(i+1) + " of " + str(len(sample_folders)), end="\r")

    sample_folder = root_path + sample_f +"\\"
    
    current_experiment = experiment(sample_folder,Specialtype="SciAps")
    sg = current_experiment.mean_signal
    
    list_of_experiments.append(current_experiment)
    
    sample_ids.append(sample_f)

In [4]:
index = 10

subplots()
current_experiment = list_of_experiments[index]

current_signal = current_experiment.mean_signal

wavelengths = current_signal.wavelengths
spectrum = current_signal.spectrum

plot(np.ndarray.flatten(np.array(wavelengths)),np.ndarray.flatten(np.array(spectrum)),'-')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 0. Pre-Processing

In [5]:
#list of processed experiments
list_of_signals_pro = []

for i in range(0,len(list_of_experiments)):
    
    print(" Removing baseline " + str(i+1) + " of " + str(len(list_of_experiments)), end = "\r")
    
    sg = list_of_experiments[i].mean_signal
    sg.remove_baseline()
    
    list_of_signals_pro.append(sg)

In [6]:
#print(list_of_signals_pro)

subplots()
#plot(np.ndarray.flatten(np.array(wavelengths)),np.ndarray.flatten(np.array(spectrum)),'-')


wavelengths = list_of_signals_pro[index].wavelengths
spectrum = list_of_signals_pro[index].spectrum



plot(np.ndarray.flatten(np.array(wavelengths)),np.ndarray.flatten(np.array(spectrum)),'-')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
import sklearn
from sklearn.preprocessing import *

#turn experiment into an array

list_of_spectra = []
for i in range(0,len(list_of_signals_pro)):
    list_of_spectra.append(np.ndarray.flatten(list_of_signals_pro[i].spectrum))
list_of_spectra = np.array(list_of_spectra)

#peak area calculation

def area_intensity(line, wavelengths, spectra):
    
    ratio_of_maximum = 0.5

    radius = 0.5
    
    lines = []
    
    for i in range(0, len(spectra)):
        intensity = get_peak_area(line, ratio_of_maximum, wavelengths, spectra[i], radius, False)
        lines.append(intensity)
    
    return lines

#standard scaler

def scaler_norm(data):
    scaler = StandardScaler()
    scaler.fit(data)
    new_data = scaler.transform(data)
    return new_data

#Normalize to a selected peak area

def internal_ref_norm(wavelengths, spectra, line):
    peak_areas = area_intensity(line, wavelengths, spectra)
    
    norm_spectra = []
    
    for k in range(len(spectra)):
        #norm_spectrum = spectra[0]
        
        #for i in range(0,len(spectra[k])):
            #norm_spectrum[i] = spectra[k][i]/peak_areas[k]
            
        norm_spectra.append(spectra[k]/peak_areas[k])
        #print(norm_spectra)
    return norm_spectra
    
#Normalize to total area

def total_area_norm(wavelengths, spectra):
    norm_spectra = []
    print(len(spectra))
    for k in range(len(spectra)):
        area = 0
        
        #norm_spectrum = spectra[0]
        #print(len(wavelengths))
        
        for i in range(1, len(wavelengths)):
            area += (wavelengths[i] - wavelengths[i-1]) * (spectra[k][i] + spectra[k][i-1])/2
            
        #print(area)
        #for j in range(0, len(spectra[k])):
            #norm_spectrum[j] = spectra[k][j]/area
            
        
        norm_spectra.append(spectra[k]/area)
    return norm_spectra

#Normalize to peak maximum

def maximum_norm(spectra):
    norm_spectra = []
    for k in range(len(spectra)):
        maximum = max(spectra[k])
        
        #norm_spectrum = spectra[0]
        #for i in range(0,len(spectra[k])):
            #norm_spectrum[i] = spectra[k][i]/maximum
            
        norm_spectra.append(spectra[k]/maximum)
    return norm_spectra



In [8]:
list_of_norm_signals = scaler_norm(list_of_spectra)

subplots()


wavelengths = list_of_signals_pro[index].wavelengths
spectrum = list_of_norm_signals[0]



plot(np.ndarray.flatten(np.array(wavelengths)),np.ndarray.flatten(np.array(spectrum)),'-')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
#print(list_of_spectra)
list_of_norm_signals = total_area_norm(wavelengths[0], list_of_spectra)

subplots()


wavelengths = list_of_signals_pro[index].wavelengths
spectrum = list_of_norm_signals[0]



plot(np.ndarray.flatten(np.array(wavelengths)),np.ndarray.flatten(np.array(spectrum)),'-')

124


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
#print(list_of_spectra)
list_of_norm_signals = maximum_norm( list_of_spectra)

subplots()


wavelengths = list_of_signals_pro[index].wavelengths
spectrum = list_of_norm_signals[0]



plot(np.ndarray.flatten(np.array(wavelengths)),np.ndarray.flatten(np.array(spectrum)),'-')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
#print(list_of_spectra)
line = 656

list_of_norm_signals = internal_ref_norm(wavelengths[0], list_of_spectra, line)

subplots()


wavelengths = list_of_signals_pro[index].wavelengths
spectrum = list_of_norm_signals[0]



plot(np.ndarray.flatten(np.array(wavelengths)),np.ndarray.flatten(np.array(spectrum)),'-')

spectrum = list_of_norm_signals[100]

plot(np.ndarray.flatten(np.array(wavelengths)),np.ndarray.flatten(np.array(spectrum)),'-', color = "red")

spectrum = list_of_norm_signals[50]

plot(np.ndarray.flatten(np.array(wavelengths)),np.ndarray.flatten(np.array(spectrum)),'-', color = "green")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 0.1 Set Target Values

In [12]:
targets = []
for i in range(0, len(sample_ids)):
    target = concentration_data3.loc[" " + str(sample_ids[i]),"Li"]
    targets.append(target)
targets = np.array(targets)[5:]
list_of_signals_pro = list_of_signals_pro[5:]

In [13]:
subplots()
plot(targets,'o', ms = 2,ls=':', lw = 0.5)
xlabel('depth (m)')
ylabel('Li (ppm)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Li (ppm)')

## 1. Univariate Linear Regression

In [14]:
j = 90
current_signal = list_of_signals_pro[j]

wavelength = np.ndarray.flatten(np.array(current_signal.wavelengths))
spectrum = np.ndarray.flatten(np.array(current_signal.spectrum))

line = 610.2

ratio_of_maximum = 0.5

radius = 0.5

subplots()

intensity = get_peak_area(line, ratio_of_maximum, wavelength, spectrum, radius, True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
lines = [610.2, 670.7]
#line = 670.7

ratio_of_maximum = 0.5

radius = 0.5

line_list = []

        

for i in range(len(lines)):

    line = []
    for j in range(0, len(list_of_signals_pro)):
        current_signal = list_of_signals_pro[j]

        wavelength = np.ndarray.flatten(np.array(current_signal.wavelengths))
        spectrum = np.ndarray.flatten(np.array(current_signal.spectrum))

        intensity = get_peak_area(lines[i], ratio_of_maximum, wavelength, spectrum, radius, False)
        line.append(intensity)
    line_list.append(line)
    #print(line)
    #print(line_list)

line_1 = line_list[0]
line_2 = line_list[1]
#print(line_1)

In [16]:
subplots()
subplot(211)
plot(line_1, 'o', ms = 1,ls=':', lw = 0.5)
xlabel('depth (m)')
ylabel("Intensity (counts)")

ax=gca()
ax2 = twinx()

plot(targets, 'o', ms = 1,ls=':', lw = 0.5, color = 'red')
xlabel('depth (m)')
ylabel("Li (ppm)")

subplot(212)

plot(targets, line_1, 'o', ms = 1)
xlabel('Li (ppm)')
ylabel("Intensity (counts)")

tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
import sklearn


#import the linear model from sklearn library
from sklearn import linear_model

#define the model
univariate_model = sklearn.linear_model.LinearRegression()

#fit the model
X = np.array(line_1)
y = np.array(targets)
x_train = X.reshape(-1,1)

y_train = y
univariate_model.fit(x_train,y_train)

#predictions
y_predictions = univariate_model.predict(x_train)

#plot the results
subplots()
plot(x_train,y_train,'o',label='True Data')
plot(x_train,y_predictions,'x',label='Predictions')
xlabel(r"$x_0$")
ylabel(r"$y$")
title("Univariate Linear Regression")
legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
from sklearn.model_selection import train_test_split

test_size = 0.16
random_state = 0
#X_train, X_test, y_train, y_test = train_test_split(X.reshape(-1,1), y, test_size=test_size, random_state=random_state)

spec=[]
print(wavelengths)
for j in range(0, len(list_of_signals_pro)):
    current_signal = list_of_signals_pro[j]
    spectrum = np.ndarray.flatten(np.array(current_signal.spectrum))
    spec.append(np.array(spectrum))
    
y = np.array(targets)
    
test_size = 0.16
random_state = 0

X_train, X_test, y_train, y_test = train_test_split(np.array(spec), y, test_size=test_size,
                                                    random_state=random_state,shuffle = True)

univariate_model.fit(X_train,y_train)

#predictions
predicted_test = univariate_model.predict(X_test)
predicted_train = univariate_model.predict(X_train)

def prev_univ(x_train, y_train, x_test, line):
    #print(x_train)
    
    #update x_train and x_test to get area of the peaks instead of intensity
    
    x_train = np.array(area_intensity(line, wavelengths[0], x_train)).reshape(-1,1)
    x_test = np.array(area_intensity(line, wavelengths[0], x_test)).reshape(-1,1)
    
    #print(x_train)
    
    univariate_model = sklearn.linear_model.LinearRegression()

    univariate_model.fit(x_train,y_train)
    
    return univariate_model.predict(x_train), univariate_model.predict(x_test)

#predictions
y_predictions_train, y_predictions_test = prev_univ(X_train, y_train, X_test, 610.2)

#plot the results
subplots()
plot(y_train,y_predictions_train,'o',color = 'r', markersize=4,label = 'Train Dataset')
plot(y_test,y_predictions_test,'x',color = 'grey', markersize=4, label = 'Test Dataset')
 
xlabel('True value (ppm)')
ylabel('Predicted value (ppm)')
legend()

[array([180.        , 180.03333333, 180.06666667, ..., 960.93333333,
       960.96666667, 961.        ])]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 1.1 Minimum error

In [19]:
from core.errors import *

#line = 610.2

#function to find peaks in throughout all spectra, returns wavelengths

def line_finder(wavelengths, X_train, percentage_of_maximum = 1, Plot = False):
    
    #average of all spectra
    
    specsum = np.zeros(len(X_train[0,:]))
    
    for i in range(len(X_train)):
        
        specsum += X_train[i,:]
        
    specsum /= len(X_train)
    
    #minimum intensity value for threshold based on percentage of max intensity
    
    threshold = max(specsum) * percentage_of_maximum / 100
        
    #peaks = signal.find_peaks(specsum, threshold = threshold)
    
    peaks = signal.find_peaks(specsum, prominence = threshold)
    
    if Plot:
        subplots()
        plot(wavelengths,specsum)
        plot(wavelengths[peaks[0]],specsum[peaks[0]],'o')
    
    return wavelengths[peaks[0]]

#function to calculate absolute error for univariate regression for given lines




def error_uv(X_train, y_train, X_test, y_test, lines):
    
    #print(lines)
    abs_error = []
    
    
    #cycle to obtain predictions and errors for each line
    
    for i in lines:
        #print(i, end = "\r")
        y_predictions_train, y_predictions_test = prev_univ(X_train, y_train, X_test, i)

        MAE = mean_absolute_error(y_test,y_predictions_test)
        #print(MAE)
        abs_error.append(MAE)
        
    return lines, abs_error



In [20]:
lines = line_finder(wavelengths[0], X_train)

abs_error = error_uv(X_train, y_train, X_test, y_test, lines)
#print(abs_error)
#mean_absolute_error(y_test,y_predictions)
subplots()
plot(abs_error[0], abs_error[1],'x')
xlabel("lines")
ylabel("mean absolute error")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'mean absolute error')

## 2. Multivariate Linear Regression

In [21]:
#define the model


#fit the model
#print(line_1)
#print(line_2)
#X = np.concatenate((np.array(line_1).reshape(-1,1),np.array(line_2).reshape(-1,1)), axis = 1)
#y = np.array(targets)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

#predictions

def prev_multiv(x_train, y_train, x_test, threshold = 1):
    #print(x_train)
    #print(len(x_train))
    #print(len(x_train[0]))
    x_train_holder = []
    x_test_holder = []
    
    lines = line_finder(wavelengths[0], x_train, percentage_of_maximum = threshold, Plot = True)
    #n_lines = len(lines)
    #x_train_holder = np.zeros(len(x_train),len(lines))
    #x_test_holder = np.zeros(len(x_train),len(lines))
    
    for i in range(len(lines)):
        #print(lines[i])
        x_train_holder.append(area_intensity(lines[i], wavelengths[0], x_train))
        x_test_holder.append(area_intensity(lines[i], wavelengths[0], x_test))
    
    x_train = np.transpose(np.array(x_train_holder))
    
    x_test = np.transpose(np.array(x_test_holder))
    
    
    #print(x_train)
    #print(len(x_train))
    #print(len(x_train[0]))
    #print(len(y_train))
    multivariate_model = sklearn.linear_model.LinearRegression()

    multivariate_model.fit(x_train,y_train)
    
    return multivariate_model.predict(x_train), multivariate_model.predict(x_test) #, n_lines

#predictions
y_predictions_train, y_predictions_test = prev_multiv(X_train, y_train, X_test, 11) #, n_lines = prev_multiv(X_train, y_train, X_test, 11)

#plot the results
subplots()
plot(y_train,y_predictions_train,'o')
plot(y_test,y_predictions_test,'x')
lims = [np.min([y_train,y_predictions_train]),np.max([y_train,y_predictions_train])]
plot(lims, lims)
xlabel('True Values')
ylabel('Predicted Values')
title("Multivariate Linear Regression")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Multivariate Linear Regression')

### 2.1 Minimum error

In [22]:
def error_mv(X_train, y_train, X_test, y_test, threshold):
    threshold_range = range(threshold, 21)
    
    #print(lines)
    abs_error = []
    #print(wavelengths[0,i])
    #n_lines_list = []
    for i in threshold_range:
        #lines = line_finder(wavelengths[0], X_train, threshold)
        print(str(threshold_range.index(i)+1) + " of " + str(len(threshold_range)), end = "\r")
        y_predictions_train, y_predictions_test = prev_multiv(X_train, y_train, X_test, i)#, n_lines = prev_multiv(X_train, y_train, X_test, i)
        #n_lines_list.append(n_lines)
        MAE = mean_absolute_error(y_test,y_predictions_test)
        #print(MAE)
        abs_error.append(MAE)
    return threshold_range, abs_error #, n_lines_list


In [23]:
abs_error = error_mv(X_train, y_train, X_test, y_test,1)
#print(abs_error)
#mean_absolute_error(y_test,y_predictions)
subplots()
plot(abs_error[0], abs_error[1],'x')
xlabel("threshold")
ylabel("mean absolute error")

#subplots()
#plot(abs_error[2], abs_error[1],'x')
#xlabel("number of lines")
#ylabel("mean absolute error")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'mean absolute error')

In [24]:
min(abs_error[1])
abs_error[0][abs_error[1].index(min(abs_error[1]))]

11

## 3.Ridge Regression

In [25]:
spec=[]

for j in range(0, len(list_of_signals_pro)):
    current_signal = list_of_signals_pro[j]
    spectrum = np.ndarray.flatten(np.array(current_signal.spectrum))
    spec.append(np.array(spectrum))
    
y = np.array(targets)
    
test_size = 0.16
random_state = 0


X_train, X_test, y_train, y_test = train_test_split(np.array(spec), y, test_size=test_size,
                                                    random_state=random_state,shuffle = True)
#define the model

def prev_ridge(x_train, y_train, x_test, a):

    ridge_model = sklearn.linear_model.Ridge(alpha = a) #100000)

    ridge_model.fit(x_train,y_train) #X_train,y_train)
    
    return ridge_model.predict(x_train), ridge_model.predict(x_test)

#predictions
y_predictions_train, y_predictions_test = prev_ridge(X_train, y_train, X_test, 100000)

#plot the results
subplots()
plot(y_train,y_predictions_train,'o')
plot(y_test,y_predictions_test,'x')
xlabel('True Values')
ylabel('Predicted Values')
title("Ridge Regression")

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Ridge Regression')

### 3.1 Minimum error

In [26]:
from core.errors import *

alpha = 10000000

def error_rr(X_train, y_train, X_test, y_test, alpha):
    #print(alpha//1000)
    alpha_range = range(0,alpha, alpha//1000)
    abs_error = []
    for i in alpha_range:
        #print(i, end = "\r")
        y_predictions_train, y_predictions_test = prev_ridge(X_train, y_train, X_test, i)

        MAE = mean_absolute_error(y_test,y_predictions_test)
        #print(MAE)
        abs_error.append(MAE)
    return alpha_range, abs_error

abs_error = error_rr(X_train, y_train, X_test, y_test, alpha)
#print(abs_error)
#mean_absolute_error(y_test,y_predictions)
subplots()
plot(abs_error[0], abs_error[1],'x')
xlabel("alpha")
ylabel("mean absolute error")

C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=7.40079e-20): result may not be accurate.
  overwrite_a=False)
C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'mean absolute error')

## 4. Principal Components Regression

In [27]:
from sklearn import decomposition
from sklearn import preprocessing
from sklearn import pipeline

# Principle components regression
def prev_pca(x_train, y_train, x_test, n):

    
    steps = [
        ('scale', sklearn.preprocessing.StandardScaler()),
        ('pca', sklearn.decomposition.PCA()),
        ('estimator', sklearn.linear_model.LinearRegression())
    ]

    pipe = sklearn.pipeline.Pipeline(steps)
    
    pca_model = pipe.set_params(pca__n_components = n) #100000)

    pca_model.fit(x_train,y_train) #X_train,y_train)
    
    return pca_model.predict(x_train), pca_model.predict(x_test)

y_predictions_train, y_predictions_test = prev_pca(X_train, y_train, X_test, 2)


#plot the results
subplots()
plot(y_train,y_predictions_train,'o')
plot(y_test,y_predictions_test,'x')
xlabel('True Values')
ylabel('Predicted Values')
title("Principal component analysis")


C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Principal component analysis')

### 4.1 Minimum error

In [28]:
abs_error = []

pca_n_components = 100


def error_pca(X_train, y_train, X_test, y_test, pca_n_components):
    #print(alpha//1000)
    pca_n_components_range = range(2,pca_n_components)
    abs_error = []

    for n in pca_n_components_range:

        y_predictions_train, y_predictions_test = prev_pca(X_train, y_train, X_test, n)

        MAE = mean_absolute_error(y_test,y_predictions_test)
        #print(MAE)
        abs_error.append(MAE)
    return pca_n_components_range, abs_error

abs_error = error_pca(X_train, y_train, X_test, y_test, pca_n_components)
    
subplots()
plot(abs_error[0],abs_error[1],'x')
xlabel("pca n components")
ylabel("mean absolute error")

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'mean absolute error')

## 5. Partial Least Squares 

In [29]:
from sklearn import cross_decomposition

#pls_model = sklearn.cross_decomposition.PLSRegression(n_components=2)

def prev_pls(x_train, y_train, x_test, n):

    pls_model = sklearn.cross_decomposition.PLSRegression(n_components = n)
    
    pls_model.fit(x_train,y_train)

    
    return pls_model.predict(x_train), pls_model.predict(x_test)



y_predictions_train, y_predictions_test = prev_pls(X_train, y_train, X_test, 10)

#plot the results
subplots()
plot(y_train,y_predictions_train,'o')
plot(y_test,y_predictions_test,'o')
xlabel('True Values')
ylabel('Predicted Values')
title("Partial Least Squares")



C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Partial Least Squares')

### 5.1 Minimum error

In [30]:
abs_error = []

n_components = 10

def error_pls(X_train, y_train, X_test, y_test, n_components):
    
    n_components_range = range(2, n_components)
    abs_error = []

    for n in n_components_range:

        y_predictions_train, y_predictions_test = prev_pls(X_train, y_train, X_test, n)

        MAE = mean_absolute_error(y_test,y_predictions_test)
        #print(MAE)
        abs_error.append(MAE)
        
    return n_components_range, abs_error

abs_error = error_pls(X_train, y_train, X_test, y_test, n_components)
    
subplots()
plot(abs_error[0],abs_error[1],'x')

xlabel("pls n components")
ylabel("mean absolute error")

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'mean absolute error')

## 6. Multilayer perceptron

In [31]:
from sklearn import neural_network

#define the model
#mlp_model = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=40)

#fit the model
def prev_mlp(x_train, y_train, x_test, n):

    mlp_model = sklearn.neural_network.MLPRegressor(hidden_layer_sizes = n)
    
    mlp_model.fit(x_train,y_train)

    
    return mlp_model.predict(x_train), mlp_model.predict(x_test)


#predictions
y_predictions_train, y_predictions_test = prev_mlp(X_train, y_train, X_test, 40)

#plot the results
subplots()
plot(y_train,y_predictions_train,'o')
plot(y_test,y_predictions_test,'o')
xlabel('True Values')
ylabel('Predicted Values')
title("Partial Least Squares")

C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Partial Least Squares')

### 6.1 Minimum error

In [32]:
abs_error = []

layer_sizes = 100

def error_mlp(X_train, y_train, X_test, y_test, layer_sizes):
    
    layer_sizes_range = range(1, layer_sizes)
    abs_error = []

    for n in layer_sizes_range:

        y_predictions_train, y_predictions_test = prev_mlp(X_train, y_train, X_test, n)

        MAE = mean_absolute_error(y_test,y_predictions_test)
        #print(MAE)
        abs_error.append(MAE)
        
    return layer_sizes_range, abs_error



In [33]:
abs_error = error_pls(X_train, y_train, X_test, y_test, layer_sizes)
    
subplots()
plot(abs_error[0],abs_error[1],'x')

xlabel("hidden layer sizes")
ylabel("mean absolute error")

C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:292: UserWarning: Y residual constant at iteration 22
  warnings.warn('Y residual constant at iteration %s' % k)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:292: UserWarning: Y residual constant at iteration 22
  warnings.warn('Y residual constant at iteration %s' % k)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:292: UserWarning: Y residual constant at iteration 22
  warnings.warn('Y residual constant at iteration %s' % k)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:292: UserWarning: Y residual constant at iteration 22
  warnings.warn('Y residual constant at iteration %s' % k)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:292: UserWarning: Y residual constant at iteration 22
  warnings.warn('Y residual constant at iteration %s' % k)
C:\Users\Miguel\Anaconda3\lib\site-

C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:292: UserWarning: Y residual constant at iteration 22
  warnings.warn('Y residual constant at iteration %s' % k)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:292: UserWarning: Y residual constant at iteration 22
  warnings.warn('Y residual constant at iteration %s' % k)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:292: UserWarning: Y residual constant at iteration 22
  warnings.warn('Y residual constant at iteration %s' % k)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:292: UserWarning: Y residual constant at iteration 22
  warnings.warn('Y residual constant at iteration %s' % k)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:292: UserWarning: Y residual constant at iteration 22
  warnings.warn('Y residual constant at iteration %s' % k)
C:\Users\Miguel\Anaconda3\lib\site-

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'mean absolute error')

In [34]:
subplots()
plot(layer_sizes,abs_error,'x')
xlabel("hidden layer sizes")
ylabel("mean absolute error")

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: x and y must have same first dimension, but have shapes (1,) and (2, 98)

## 7. Cross-validation

In [35]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=6, shuffle = True)


def train_test_split(X,y):
    data_lists = []
    for train_index, test_index in kf.split(X):
        #print(train_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        data_lists.append([X_train, y_train, X_test, y_test])
        
    return data_lists

models_tested = [prev_univ,prev_multiv,prev_ridge,prev_pca,prev_pls,prev_mlp]

parameters = [0, 0, 100000, 2, 10, 40]


evaluation_list = [mean_absolute_error, mean_square_error, root_mean_square_error,
                  mean_absolute_percentage_error, max_error, max_squared_error,
                  max_percentage_error, r_squared]

text = ["Models", "Mean absolute error", "Mean square error", "Root mean square error",
       "Mean absolute percentage error (%)", "Max error", "Max squared error",
       "Max percentage error", "R^2"]

def test_all_models(x, y, x_test, y_test, models = models_tested, parameters = parameters,
                    evaluation_list = evaluation_list):
    
    data_train = []
    
    data_test = []
    
    
    for i in range(len(models)):
        current_line_train = []
        current_line_train.append(models[i])
        
        current_line_test = []
        current_line_test.append(models[i])
        
        #predictions
        
        if i <= 1:
            if i==0:
                y_predictions_train, y_predictions_test = models[i](x[:,0].reshape(-1,1), y, x_test[:,0].reshape(-1,1))
            else:
                y_predictions_train, y_predictions_test = models[i](x, y, x_test)

        else:
            y_predictions_train, y_predictions_test = models[i](x, y, x_test, parameters[i])
            
        
        #print(models[i])
        
        
        for j in range(len(evaluation_list)):
            err_train = evaluation_list[j](y_test, y_predictions_train)
            
            err_test = evaluation_list[j](y_test, y_predictions_test)
            
            current_line_train.append(err_train)
            current_line_train.append(err_test)
            #print(text[j] + ' -> ', err)
        data_train.append(current_line_train)
        data_test.append(current_line_test)
        #print("\n")
    return data_train, data_test



In [36]:
X = np.array(spec)

data_lists = train_test_split(X,y)

### 7.-1 Univariate minimum error

In [37]:
lines = line_finder(wavelengths[0], X_train)

error_lists = []

for j in range(len(data_lists)):
    lines, errors = error_uv(data_lists[j][0], data_lists[j][1], data_lists[j][2], data_lists[j][3], lines)
    error_lists.append(errors)


In [38]:
mean_error_list = []

subplots()

for k in range(len(lines)):
    mean_error = 0

    for m in range(len(data_lists)):
        mean_error += error_lists[m][k]
        
    mean_error_list.append(mean_error/len(data_lists))

for m in range(len(data_lists)):
    print(error_lists[m].index(min(error_lists[m])))
    plot(lines, error_lists[m])
    
print(mean_error_list.index(min(mean_error_list)))
    
subplots()

plot(lines, mean_error_list, "x")

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

77
76
77
63
77
87
77


C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
min_error_line = lines[mean_error_list.index(min(mean_error_list))]
print(min_error_line)

parameters[0] = min_error_line

610.3000000001099


### 7.0 Multivariate minimum error

In [ ]:
threshold = 1

error_lists = []

for j in range(len(data_lists)):
    thresholds, errors = error_mv(data_lists[j][0], data_lists[j][1], data_lists[j][2], data_lists[j][3], threshold)
    error_lists.append(errors)


C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
mean_error_list = []

subplots()

for k in range(len(thresholds)):
    mean_error = 0

    for m in range(len(data_lists)):
        mean_error += error_lists[m][k]
        
    mean_error_list.append(mean_error/len(data_lists))

for m in range(len(data_lists)):
    print(error_lists[m].index(min(error_lists[m])))
    plot(thresholds, error_lists[m])
    
print(mean_error_list.index(min(mean_error_list)))
    
subplots()

plot(thresholds, mean_error_list, "x")

In [ ]:
min_error_threshold = thresholds[mean_error_list.index(min(mean_error_list))]
print(min_error_threshold)

parameters[0] = min_error_threshold

### 7.1 Ridge minimum error

In [78]:
alpha = 20000000

#print(len(data_lists))

#for i in range(len(models_tested[2:])):
error_lists = []
for j in range(len(data_lists)):
    #print(data_lists[j])
    alphas, errors = error_rr(data_lists[j][0], data_lists[j][1], data_lists[j][2], data_lists[j][3], alpha)
    error_lists.append(errors)
    



C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=5.48361e-18): result may not be accurate.
  overwrite_a=False)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=3.19428e-19): result may not be accurate.
  overwrite_a=False)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=4.28521e-19): result may not be accurate.
  overwrite_a=False)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matri

In [79]:


mean_error_list = []

subplots()

for k in range(len(alphas)):
    mean_error = 0

    for m in range(len(data_lists)):
        mean_error += error_lists[m][k]
        
    mean_error_list.append(mean_error/len(data_lists))

for m in range(len(data_lists)):
    print(error_lists[m].index(min(error_lists[m])))
    plot(alphas, error_lists[m])
    
print(mean_error_list.index(min(mean_error_list)))
    
subplots()

plot(alphas, mean_error_list, "x")


C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

999
999
999
709
339
36
709


C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [126]:
min_error_alpha = alphas[mean_error_list.index(min(mean_error_list))]
print(min_error_alpha)

parameters[2] = min_error_alpha


60000


### 7.2 PCA minimum error

In [81]:
pca_n_components = 100

#print(len(data_lists))

#for i in range(len(models_tested[2:])):
error_lists = []
for j in range(len(data_lists)):
    #print(data_lists[j])
    pca_n_components_list, errors = error_pca(data_lists[j][0], data_lists[j][1], data_lists[j][2], data_lists[j][3], pca_n_components)
    error_lists.append(errors)
    


In [82]:
mean_error_list = []

subplots()

for k in range(len(pca_n_components_list)):
    mean_error = 0

    for m in range(len(data_lists)):
        mean_error += error_lists[m][k]
        
    mean_error_list.append(mean_error/len(data_lists))

for m in range(len(data_lists)):
    print(error_lists[m].index(min(error_lists[m])))
    plot(pca_n_components_list, error_lists[m])
    
print(mean_error_list.index(min(mean_error_list)))
    
subplots()

plot(pca_n_components_list, mean_error_list, "x")

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

6
75
28
54
97
77
23


C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [120]:
min_error_pca_n_components = pca_n_components_list[mean_error_list.index(min(mean_error_list))]
print(min_error_pca_n_components)

parameters[3] = min_error_pca_n_components


66


### 7.3 PLS minimum error

In [123]:
n_components = 10

error_lists = []
for j in range(len(data_lists)):
    #print(len(data_lists[j][0]))
    #print(len(data_lists[j][1]))
    n_components_list, errors = error_pls(data_lists[j][0], data_lists[j][1], data_lists[j][2],
                                          data_lists[j][3], n_components)
    error_lists.append(errors)
    


In [124]:


mean_error_list = []

subplots()

for k in range(len(n_components_list)):
    mean_error = 0

    for m in range(len(data_lists)):
        mean_error += error_lists[m][k]
        
    mean_error_list.append(mean_error/len(data_lists))

for m in range(len(data_lists)):
    print(error_lists[m].index(min(error_lists[m])))
    plot(n_components_list, error_lists[m])
    
print(mean_error_list.index(min(mean_error_list)))
    
subplots()

plot(n_components_list, mean_error_list, "x")

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2
4
0
3
3
2
3


C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [125]:
print(min_error_n_components)
min_error_n_components = n_components_list[mean_error_list.index(min(mean_error_list))]
print(min_error_n_components)

parameters[4] = min_error_n_components

5
5


### 7.4 Multilayer perceptron minimum error

In [90]:
layer_sizes = 100

error_lists = []
for j in range(len(data_lists)):
    #print(len(data_lists[j][0]))
    #print(len(data_lists[j][1]))
    layer_sizes_list, errors = error_mlp(data_lists[j][0], data_lists[j][1], data_lists[j][2],
                                          data_lists[j][3], layer_sizes)
    error_lists.append(errors)


C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Conv

C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Conv

C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Conv

C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Conv

C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Conv

...

C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Conv

C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Conv

C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Conv

C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Conv

C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Conv

In [91]:
mean_error_list = []

subplots()

for k in range(len(layer_sizes_list)):
    mean_error = 0

    for m in range(len(data_lists)):
        mean_error += error_lists[m][k]
        
    mean_error_list.append(mean_error/len(data_lists))

for m in range(len(data_lists)):
    print(error_lists[m].index(min(error_lists[m])))
    plot(layer_sizes_list, error_lists[m])
    
print(mean_error_list.index(min(mean_error_list)))
    
subplots()

plot(layer_sizes_list, mean_error_list, "x")

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2
27
96
64
69
84
64


C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [127]:
min_error_layer_sizes = layer_sizes_list[mean_error_list.index(min(mean_error_list))]
print(min_error_layer_sizes)

parameters[5] = min_error_layer_sizes

4


### ???

In [46]:
def test_all_models(x, y, x_test, y_test, models = models_tested, parameters = parameters,
                    evaluation_list = evaluation_list):
    
    data_train = []
    
    data_test = []
    
    
    for i in range(len(models)):
        current_line_train = []
        current_line_train.append(models[i])
        
        current_line_test = []
        current_line_test.append(models[i])
        
        #predictions
        
        if i <= 1:
            if i==0:
                y_predictions_train, y_predictions_test = models[i](x[:,0].reshape(-1,1), y, x_test[:,0].reshape(-1,1))
            else:
                y_predictions_train, y_predictions_test = models[i](x, y, x_test)
                
        else:
            y_predictions_train, y_predictions_test = models[i](x, y, x_test, parameters[i])
            
        
        #print(models[i])
        
        
        for j in range(len(evaluation_list)):
            err_train = evaluation_list[j](y_test, y_predictions_train)
            
            err_test = evaluation_list[j](y_test, y_predictions_test)
            
            current_line_train.append(err_train)
            current_line_test.append(err_test)
            #print(text[j] + ' -> ', err)
        data_train.append(current_line_train)
        data_test.append(current_line_test)
        #print("\n")
    return data_train, data_test

In [130]:
data_train = []
data_test = []

for j in range(len(data_lists)):
    d_train, d_test = test_all_models(data_lists[j][0], data_lists[j][1], data_lists[j][2],
                                          data_lists[j][3], parameters = parameters)
    data_train.append(d_train)
    data_test.append(d_test)
    
aver_data_train = data_train[0]
for i in range(1,len(data_train[0])):
    for j in range(1,len(data_train[0][0])):
        value = 0
        for k in range(len(data_train)):
            value += data_train[k][i][j]
        aver_data_train[i][j] = value/len(data_train)
        
aver_data_test = data_test[0]
for i in range(1,len(data_test[0])):
    for j in range(1,len(data_test[0][0])):
        value = 0
        for k in range(len(data_test)):
            value += data_test[k][i][j]
        aver_data_test[i][j] = value/len(data_test)


C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Miguel\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Conv

In [131]:
#print(data_train)
error_table_train = pd.DataFrame(data_train[0], columns = text)
error_table_train

,Models,Mean absolute error,Mean square error,Root mean square error,Mean absolute percentage error (%),Max error,Max squared error,Max percentage error,R^2
0,<function prev_univ at 0x000002A46B7CE3A8>,2030.05,5.7245e+06,2392.59,238.105,4112.89,1.69159e+07,10.9185,0.66896
1,<function prev_multiv at 0x000002A46B814168>,3796.62,2.30731e+07,4792.07,255.701,9916.67,9.98336e+07,19.6219,-7.71086
2,<function prev_ridge at 0x000002A46B8140D8>,3628.22,2.08958e+07,4556.42,253.893,9389.67,9.04751e+07,18.7229,-1.3153
3,<function prev_pca at 0x000002A46B99B8B8>,3692.11,2.17474e+07,4629.42,275.111,9914,1.03413e+08,17.3773,-0.0572472
4,<function prev_pls at 0x000002A41212C948>,[3787.0663912835475],[22795823.5678188],[4759.682499457],[263.4072301841398],[9994.345388190019],[102816394.06590372],[20.252753568779237],[-5.659522269644938]
5,<function prev_mlp at 0x000002A4000B9048>,3127.28,1.60523e+07,3936.56,219.722,8265.79,7.30514e+07,14.0987,-5.66695


In [132]:
error_table_train = pd.DataFrame(data_train[1], columns = text)
error_table_train

,Models,Mean absolute error,Mean square error,Root mean square error,Mean absolute percentage error (%),Max error,Max squared error,Max percentage error,R^2
0,<function prev_univ at 0x000002A46B7CE3A8>,3227.29,1.45361e+07,3812.62,455.426,8318.03,6.91896e+07,19.7506,0.041849
1,<function prev_multiv at 0x000002A46B814168>,4156.15,2.92576e+07,5409.03,441.94,11610,1.34792e+08,34.9447,0.193846
2,<function prev_ridge at 0x000002A46B8140D8>,3872.92,2.71135e+07,5207.06,445.34,11445,1.30987e+08,33.0828,0.942084
3,<function prev_pca at 0x000002A46B99B8B8>,4503.51,3.1489e+07,5611.5,515.462,11780.7,1.38786e+08,29.7981,0.998819
4,<function prev_pls at 0x000002A41212C948>,[4175.284131794693],[29830519.599914443],[5461.7322892937955],[457.64897070470835],[12271.028872825711],[150578149.59772223],[36.07697552258187],[0.470437642168851]
5,<function prev_mlp at 0x000002A4000B9048>,3694.93,2.26653e+07,4760.8,416.815,10686.3,1.14198e+08,21.3806,-1.69087


In [133]:
error_table_train = pd.DataFrame(data_train[2], columns = text)
error_table_train

,Models,Mean absolute error,Mean square error,Root mean square error,Mean absolute percentage error (%),Max error,Max squared error,Max percentage error,R^2
0,<function prev_univ at 0x000002A46B7CE3A8>,3313.07,1.57801e+07,3972.42,141.957,8055.61,6.48928e+07,7.13046,-6.28277
1,<function prev_multiv at 0x000002A46B814168>,3852,2.47625e+07,4976.19,155.908,11010,1.2122e+08,11,-33.0214
2,<function prev_ridge at 0x000002A46B8140D8>,3697.63,2.33077e+07,4827.8,152.037,10845,1.17613e+08,10.3784,-7.09496
3,<function prev_pca at 0x000002A46B99B8B8>,3734.95,2.64332e+07,5141.32,162.632,13514,1.82628e+08,12.0004,-3.17013
4,<function prev_pls at 0x000002A41212C948>,[3890.571527829802],[24772440.818522666],[4977.192061647075],[161.6984578562668],[11671.028872825711],[136212914.9503314],[11.378005674461948],[-24.596393923977622]
5,<function prev_mlp at 0x000002A4000B9048>,3708.91,2.23713e+07,4729.84,146.754,11212,1.2571e+08,9.47979,-27.6992


In [134]:
error_table_train = pd.DataFrame(data_train[3], columns = text)
error_table_train

,Models,Mean absolute error,Mean square error,Root mean square error,Mean absolute percentage error (%),Max error,Max squared error,Max percentage error,R^2
0,<function prev_univ at 0x000002A46B7CE3A8>,2756.95,9.75296e+06,3122.97,197.658,5644.58,3.18613e+07,9.45829,-0.328142
1,<function prev_multiv at 0x000002A46B814168>,3954,2.17263e+07,4661.14,173.119,8590,7.37881e+07,9.5,-5.2044
2,<function prev_ridge at 0x000002A46B8140D8>,3644.52,1.86537e+07,4319,175.763,8086.76,6.53957e+07,9.97097,-0.476259
3,<function prev_pca at 0x000002A46B99B8B8>,3567.84,1.76109e+07,4196.53,179.941,7808.97,6.098e+07,7.34021,0.0420761
4,<function prev_pls at 0x000002A41212C948>,[3951.5156434237156],[21412300.02226322],[4627.342652350615],[183.70847036418405],[8097.229714666002],[65565129.05207007],[9.718087212155234],[-3.6679543756072883]
5,<function prev_mlp at 0x000002A4000B9048>,2908.49,1.23858e+07,3519.34,162.429,5928.58,3.51481e+07,9.50226,-0.571455


In [135]:
error_table_train = pd.DataFrame(data_train[4], columns = text)
error_table_train

,Models,Mean absolute error,Mean square error,Root mean square error,Mean absolute percentage error (%),Max error,Max squared error,Max percentage error,R^2
0,<function prev_univ at 0x000002A46B7CE3A8>,2625.27,9.02216e+06,3003.69,281.661,6979.63,4.87153e+07,17.762,-0.0591308
1,<function prev_multiv at 0x000002A46B814168>,3359.5,1.89351e+07,4351.45,314.871,8170,6.67489e+07,31.5,0.994565
2,<function prev_ridge at 0x000002A46B8140D8>,3341.1,1.70285e+07,4126.56,313.411,7155.97,5.12079e+07,29.8165,0.83329
3,<function prev_pca at 0x000002A46B99B8B8>,3333.18,1.64943e+07,4061.31,320.811,7579.2,5.74443e+07,27.1158,0.411803
4,<function prev_pls at 0x000002A41212C948>,[3297.2323322485413],[18184015.355617553],[4264.271960794427],[321.8919001932356],[7852.046752791866],[61654638.20802929],[32.52376536833444],[0.9742889580050179]
5,<function prev_mlp at 0x000002A4000B9048>,2904.79,1.31736e+07,3629.54,297.726,6452.6,4.16361e+07,26.8858,0.195867


In [136]:
error_table_train = pd.DataFrame(data_train[5], columns = text)
error_table_train

,Models,Mean absolute error,Mean square error,Root mean square error,Mean absolute percentage error (%),Max error,Max squared error,Max percentage error,R^2
0,<function prev_univ at 0x000002A46B7CE3A8>,3344.72,1.57015e+07,3962.51,144.877,8294.87,6.88049e+07,7.20745,0.278185
1,<function prev_multiv at 0x000002A46B814168>,3671.05,2.15238e+07,4639.38,150.719,10210,1.04244e+08,11.7869,0.998083
2,<function prev_ridge at 0x000002A46B8140D8>,3642.88,2.08682e+07,4568.17,150.435,10045,1.00901e+08,11.1245,0.807444
3,<function prev_pca at 0x000002A46B99B8B8>,3423.04,2.04212e+07,4518.98,144.644,10797,1.16575e+08,11.1851,0.751096
4,<function prev_pls at 0x000002A41212C948>,[3617.051250675175],[21013288.379877258],[4584.0253467751745],[151.4702747802834],[10871.028872825711],[118179268.75381024],[12.189678177705353],[0.8299062335561442]
5,<function prev_mlp at 0x000002A4000B9048>,3409.74,1.86127e+07,4314.24,112.701,9130.61,8.3368e+07,6.78406,-0.204093


In [137]:
error_table_train_aver = pd.DataFrame(aver_data_train, columns = text)
error_table_train_aver

,Models,Mean absolute error,Mean square error,Root mean square error,Mean absolute percentage error (%),Max error,Max squared error,Max percentage error,R^2
0,<function prev_univ at 0x000002A46B7CE3A8>,2030.05,5.7245e+06,2392.59,238.105,4112.89,1.69159e+07,10.9185,0.66896
1,<function prev_multiv at 0x000002A46B814168>,3796.62,2.30731e+07,4792.07,255.701,9916.67,9.98336e+07,19.6219,-7.71086
2,<function prev_ridge at 0x000002A46B8140D8>,3628.22,2.08958e+07,4556.42,253.893,9389.67,9.04751e+07,18.7229,-1.3153
3,<function prev_pca at 0x000002A46B99B8B8>,3692.11,2.17474e+07,4629.42,275.111,9914,1.03413e+08,17.3773,-0.0572472
4,<function prev_pls at 0x000002A41212C948>,[3787.0663912835475],[22795823.5678188],[4759.682499457],[263.4072301841398],[9994.345388190019],[102816394.06590372],[20.252753568779237],[-5.659522269644938]
5,<function prev_mlp at 0x000002A4000B9048>,3127.28,1.60523e+07,3936.56,219.722,8265.79,7.30514e+07,14.0987,-5.66695


In [138]:
error_table_test = pd.DataFrame(data_test[0], columns = text)
error_table_test

,Models,Mean absolute error,Mean square error,Root mean square error,Mean absolute percentage error (%),Max error,Max squared error,Max percentage error,R^2
0,<function prev_univ at 0x000002A46B7CE3A8>,2213.48,6.1692e+06,2483.79,232.442,4276.35,1.82871e+07,12.7904,0.965435
1,<function prev_multiv at 0x000002A46B814168>,4160.55,2.53617e+07,5022.04,327.796,10111.6,1.03567e+08,18.021,-5.74443
2,<function prev_ridge at 0x000002A46B8140D8>,4037.24,2.29591e+07,4777.9,320.593,9166.51,8.51336e+07,17.2102,-4.92516
3,<function prev_pca at 0x000002A46B99B8B8>,3316.24,1.64651e+07,4029.14,287.917,8157.81,6.99643e+07,16.7297,-0.0494715
4,<function prev_pls at 0x000002A41212C948>,[3381.8578157866573],[16912495.53168424],[4084.1115274393974],[289.4891825090039],[8338.751293157542],[73385223.87639527],[16.17087799344618],[-0.07861991092512065]
5,<function prev_mlp at 0x000002A4000B9048>,3253.64,1.55764e+07,3915.58,263.113,7243.8,5.40061e+07,16.2124,0.011332


In [139]:
error_table_test = pd.DataFrame(data_test[1], columns = text)
error_table_test

,Models,Mean absolute error,Mean square error,Root mean square error,Mean absolute percentage error (%),Max error,Max squared error,Max percentage error,R^2
0,<function prev_univ at 0x000002A46B7CE3A8>,2910.51,1.2153e+07,3486.11,440.094,7108.42,5.05296e+07,18.4062,0.621858
1,<function prev_multiv at 0x000002A46B814168>,3786.33,1.9709e+07,4439.48,521.254,8194.14,6.7144e+07,25.6067,-1.08718
2,<function prev_ridge at 0x000002A46B8140D8>,3716.61,1.8073e+07,4251.23,500.766,7245.57,5.24983e+07,22.6424,-0.699534
3,<function prev_pca at 0x000002A46B99B8B8>,3470.65,1.79268e+07,4234,497.464,8264.36,6.82996e+07,25.8723,0.251314
4,<function prev_pls at 0x000002A41212C948>,[3460.033258917737],[18418780.44625063],[4291.710666651543],[495.06086303653444],[8758.434846017653],[76710180.95193627],[24.239084173633586],[0.3855172217751357]
5,<function prev_mlp at 0x000002A4000B9048>,3181.37,1.42845e+07,3779.49,416.385,8400.13,7.05622e+07,25.7967,-0.130397


In [140]:
error_table_test = pd.DataFrame(data_test[2], columns = text)
error_table_test

,Models,Mean absolute error,Mean square error,Root mean square error,Mean absolute percentage error (%),Max error,Max squared error,Max percentage error,R^2
0,<function prev_univ at 0x000002A46B7CE3A8>,3011.64,1.44593e+07,3802.54,124.569,8787.85,7.72263e+07,6.4596,-0.0905621
1,<function prev_multiv at 0x000002A46B814168>,4039.77,2.36855e+07,4866.77,178.102,12012.9,1.4431e+08,10.9718,-22.1979
2,<function prev_ridge at 0x000002A46B8140D8>,3941.14,2.1444e+07,4630.77,177.662,10634.6,1.13095e+08,11.1165,-16.4325
3,<function prev_pca at 0x000002A46B99B8B8>,3613.57,2.01467e+07,4488.51,157.735,9810.68,9.62495e+07,7.8058,-2.04819
4,<function prev_pls at 0x000002A41212C948>,[3657.602482596184],[20147055.688923273],[4488.547169065206],[160.38817423220962],[9688.434846017653],[93865769.76552911],[8.065641607848164],[-2.2846904254956226]
5,<function prev_mlp at 0x000002A4000B9048>,3629.68,2.05219e+07,4530.11,155.628,9180.34,8.42786e+07,8.25195,-1.79637


In [141]:
error_table_test = pd.DataFrame(data_test[3], columns = text)
error_table_test

,Models,Mean absolute error,Mean square error,Root mean square error,Mean absolute percentage error (%),Max error,Max squared error,Max percentage error,R^2
0,<function prev_univ at 0x000002A46B7CE3A8>,2938.94,1.08061e+07,3287.26,206.194,5119.97,2.62141e+07,10.2784,0.801117
1,<function prev_multiv at 0x000002A46B814168>,4906.4,3.20764e+07,5663.61,343.729,10172.9,1.03488e+08,21.2436,-3.23055
2,<function prev_ridge at 0x000002A46B8140D8>,4695.25,2.86464e+07,5352.23,327.273,8794.63,7.73454e+07,19.3006,-2.17913
3,<function prev_pca at 0x000002A46B99B8B8>,3217.56,1.47236e+07,3837.13,249.113,6047.07,3.65671e+07,12.7595,0.402336
4,<function prev_pls at 0x000002A41212C948>,[3383.4178092244847],[15741203.12557534],[3967.518509796185],[260.286280943905],[6318.434846017653],[39922618.90337013],[13.334352697981597],[0.400978704666498]
5,<function prev_mlp at 0x000002A4000B9048>,3152.25,1.44536e+07,3801.79,246.166,6875.94,4.72785e+07,13.243,0.772767


In [142]:
error_table_test = pd.DataFrame(data_test[4], columns = text)
error_table_test

,Models,Mean absolute error,Mean square error,Root mean square error,Mean absolute percentage error (%),Max error,Max squared error,Max percentage error,R^2
0,<function prev_univ at 0x000002A46B7CE3A8>,2637.89,9.15384e+06,3025.53,272.259,5723.15,3.27544e+07,14.932,0.287879
1,<function prev_multiv at 0x000002A46B814168>,4103,2.48598e+07,4985.96,360.797,9486.73,8.9998e+07,16.4488,-0.603525
2,<function prev_ridge at 0x000002A46B8140D8>,3838.42,2.15549e+07,4642.73,351.038,8908.52,7.93617e+07,16.2865,-0.426826
3,<function prev_pca at 0x000002A46B99B8B8>,3081.61,1.39824e+07,3739.3,329.568,7225.97,5.22146e+07,23.0378,0.151976
4,<function prev_pls at 0x000002A41212C948>,[3154.3406098591695],[14343895.993974596],[3787.3336259134335],[331.130196322546],[7268.434846017653],[52830145.11080367],[21.8203386069937],[0.13530947776123758]
5,<function prev_mlp at 0x000002A4000B9048>,2884.18,1.26326e+07,3554.24,318.832,5962.41,3.55503e+07,22.7341,0.310276


In [143]:
error_table_test = pd.DataFrame(data_test[5], columns = text)
error_table_test

,Models,Mean absolute error,Mean square error,Root mean square error,Mean absolute percentage error (%),Max error,Max squared error,Max percentage error,R^2
0,<function prev_univ at 0x000002A46B7CE3A8>,3318.82,1.5394e+07,3923.52,135.985,9151.27,8.37457e+07,6.06066,0.900575
1,<function prev_multiv at 0x000002A46B814168>,4250.32,2.76826e+07,5261.43,190.461,10606,1.12487e+08,8.98254,-5.9453
2,<function prev_ridge at 0x000002A46B8140D8>,4269.72,2.63106e+07,5129.38,195.934,9797.18,9.59847e+07,9.28355,-9.23835
3,<function prev_pca at 0x000002A46B99B8B8>,3784.6,2.1565e+07,4643.81,169.813,11207.3,1.25603e+08,8.86289,-0.0362764
4,<function prev_pls at 0x000002A41212C948>,[3816.7185034643626],[22233347.1029277],[4715.225032056021],[168.25056034089818],[11738.434846017653],[137790852.6342015],[7.978493878161457],[-0.08077060306168438]
5,<function prev_mlp at 0x000002A4000B9048>,3938.8,2.11165e+07,4595.27,167.108,7326.46,5.3677e+07,8.33319,0.999823


In [144]:
error_table_test_aver = pd.DataFrame(aver_data_test, columns = text)
error_table_test_aver

,Models,Mean absolute error,Mean square error,Root mean square error,Mean absolute percentage error (%),Max error,Max squared error,Max percentage error,R^2
0,<function prev_univ at 0x000002A46B7CE3A8>,2213.48,6.1692e+06,2483.79,232.442,4276.35,1.82871e+07,12.7904,0.965435
1,<function prev_multiv at 0x000002A46B814168>,4160.55,2.53617e+07,5022.04,327.796,10111.6,1.03567e+08,18.021,-5.74443
2,<function prev_ridge at 0x000002A46B8140D8>,4037.24,2.29591e+07,4777.9,320.593,9166.51,8.51336e+07,17.2102,-4.92516
3,<function prev_pca at 0x000002A46B99B8B8>,3316.24,1.64651e+07,4029.14,287.917,8157.81,6.99643e+07,16.7297,-0.0494715
4,<function prev_pls at 0x000002A41212C948>,[3381.8578157866573],[16912495.53168424],[4084.1115274393974],[289.4891825090039],[8338.751293157542],[73385223.87639527],[16.17087799344618],[-0.07861991092512065]
5,<function prev_mlp at 0x000002A4000B9048>,3253.64,1.55764e+07,3915.58,263.113,7243.8,5.40061e+07,16.2124,0.011332


In [47]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=len(targets), shuffle = False)

In [48]:
X = np.array(spec)

data_lists = train_test_split(X,y)

min_error_alpha = 60000

parameters[2] = min_error_alpha

min_error_pca_n_components = 66

parameters[3] = min_error_pca_n_components

min_error_n_components = 5

parameters[4] = min_error_n_components

min_error_layer_sizes = 4

parameters[5] = min_error_layer_sizes

In [49]:
def test_all_models(x, y, x_test, y_test, models = models_tested, parameters = parameters):
    
    data_train = []
    
    data_test = []
    
    y_list_train = []
    
    y_list_test = []
    
    
    
    for i in range(len(models)):
        current_line_train = []
        current_line_train.append(models[i])
        
        current_line_test = []
        current_line_test.append(models[i])
        
        #predictions
        
        if i <= 1:
            if i==0:
                y_predictions_train, y_predictions_test = models[i](x[:,0].reshape(-1,1), y, x_test[:,0].reshape(-1,1))
            else:
                y_predictions_train, y_predictions_test = models[i](x, y, x_test)
                
        else:
            y_predictions_train, y_predictions_test = models[i](x, y, x_test, parameters[i])
        
        #print("predictions")
        #print(len(y_predictions_train))
        #print(len(y_predictions_test))
            
        
        y_list_train.append(y_predictions_train)
    
        y_list_test.append(y_predictions_test)
        
        #print(models[i])
        
        
        for j in range(len(evaluation_list)):
            err_train = evaluation_list[j](y_test, y_predictions_train)
            
            err_test = evaluation_list[j](y_test, y_predictions_test)
            
            current_line_train.append(err_train)
            current_line_test.append(err_test)
            #print(text[j] + ' -> ', err)
        data_train.append(current_line_train)
        data_test.append(current_line_test)
        #print("\n")
    return data_train, data_test, y_list_train, y_list_test

In [59]:
data_train = []
data_test = []

y_list_train = []
y_list_test0 = []
y_list_test1 = []
y_list_test2 = []
y_list_test3 = []
y_list_test4 = []

y_list = []

for j in range(len(data_lists)):
    #print(len(data_lists[j][0]),len(data_lists[j][1]),len(data_lists[j][2]),len(data_lists[j][3]))
    d_train, d_test, y_l_train, y_l_test = test_all_models(data_lists[j][0], data_lists[j][1], np.array(data_lists[j][2]),
                                          data_lists[j][3], models = models_tested[:5], parameters = parameters)
    data_train.append(d_train)
    data_test.append(d_test)
    
    y_list_train.append(y_l_train)
    y_list_test0.append(y_l_test[0])
    y_list_test1.append(y_l_test[1])
    y_list_test2.append(y_l_test[2])
    y_list_test3.append(y_l_test[3])
    y_list_test4.append(y_l_test[4])
    
    y_list.append(data_lists[j][3])
    
#print(y_list_train)
    
"""
subplot()
    
plot(y_list, y_list_train, "x")

plot(y_list, y_list_test, "o")
"""

C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in true_divide
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeW

C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in true_divide
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeW

C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in true_divide
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeW

C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in true_divide
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeW

C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in true_divide
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in true_divide
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarn

...

C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in true_divide
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeW

C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in true_divide
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeW

C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in true_divide
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeW

C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in true_divide
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = sqerror/sqdiff
C:\Users\Miguel\Desktop\Furo SciAps\simplibs-master\core\errors.py:90: RuntimeW

'\nsubplot()\n    \nplot(y_list, y_list_train, "x")\n\nplot(y_list, y_list_test, "o")\n'

In [60]:
subplots()

plot(y_list,y_list_test2, "x")

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …